# <center>Minimum Transformation Method
## <center>An inverse model for inferring surface fluxes and mixing from gridded hydrographic TS data, <br> by Taimoor Sohail and Jan D. Zika



##### <center>We use an optimal transport method, minimzing a cost function which respects the volume and mass conservation, and the fundamental physics of mixing in TS space. <br>See also: Jan D Zika et al. (2021) [www.doi.org/10.1175/JCLI-D-20-0355.1 ] for a simpler version of this model which optimizes ocean circulation only.
------------

### __The code is laid out as follows:__
#### A) Pre-processing
i) Load necessary modules for computation \
ii) Define key parameters
#### B) Load Data
#### C) Define constraints
i) Define Connectivity \
ii) Define weights
#### D) Run optimisation
#### E) Save Output
---------

## A) Pre-processing

### i) Load necessary modules for computation

In [1]:
## Module to run the minimisation
from WM_Methods import MTM
## Module to track runtime of cells and loops
import time
from tqdm.notebook import tqdm
## Suppress warnings related to division by zero
import warnings
warnings.filterwarnings('ignore')
## Module to load files and handle array computations
import xarray as xr
import numpy as np
## Modules to plot outputs
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# Specify font properties for plots
from matplotlib import rc
font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 16}

rc('font', **font)

### ii) Define key parameters

In [2]:
# Thermal expansion, haline contraction and true scale factor
alph = 1.7657*10**-4
bet = 7.5544*10**-4

volnorming = 10**15 #normalising coeffcients
areanorming = 10**12 #normalising coeffcients
ST_scale=bet/alph

# Establish basic constants 
yr2sec = 365.25*24*60*60
Cp=4000
rho=1024
S0=35

# Range of years of which 'early' and 'late' are defined
dyrs = 10
init_early = 1970
init_late = 2005
Early_period = (np.array([init_early,init_early+dyrs]) - 1970)*12
Late_period = (np.array([init_late,init_late+dyrs]) - 1970)*12

-----------
## B) Load Data



The data is presented as monthly T-S observations from EN4, from January 1970 to December 2014. The observations have already been binned via Binary Space Partitioning according to a specific criterion. Folowing Sohail et al. (TBD) the BSP partitioning occurs in an xyxyxyx order, shown to minimize variability in the temperature and salinity signals. 



In [3]:
%%time
EN4_BSP_data = xr.open_mfdataset('BSP_processed/BSP_EN4_TS_*.nc')

## Early Period
Part_early = EN4_BSP_data.Partitions.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time')
SA_early =  EN4_BSP_data.S_mean.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time')
CT_early = EN4_BSP_data.T_mean.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time')
V_early = EN4_BSP_data.V_sum.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time')
A_early = EN4_BSP_data.A_sum.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time')


## Late Period
Part_late = EN4_BSP_data.Partitions.isel(Time=slice(Late_period[0],Late_period[1])).mean('Time')
SA_late =  EN4_BSP_data.S_mean.isel(Time=slice(Late_period[0],Late_period[1])).mean('Time')
CT_late = EN4_BSP_data.T_mean.isel(Time=slice(Late_period[0],Late_period[1])).mean('Time')
V_late = EN4_BSP_data.V_sum.isel(Time=slice(Late_period[0],Late_period[1])).mean('Time')
A_late = EN4_BSP_data.A_sum.isel(Time=slice(Late_period[0],Late_period[1])).mean('Time')

Basins = EN4_BSP_data.Basin.values

CPU times: user 3.27 s, sys: 277 ms, total: 3.55 s
Wall time: 3.89 s


In [4]:

# Flatten the early and late variables to a 1D array
Vol_1 = V_early.values.flatten()
Vol_2 = V_late.values.flatten()
S_1 = SA_early.values.flatten()-S0 # Remove reference salinity S0
S_2 = SA_late.values.flatten()-S0 # Remove reference salinity S0
T_1 = CT_early.values.flatten()
T_2 = CT_late.values.flatten()
A_1 = A_early.values.flatten()
A_2 = A_late.values.flatten()

# Do the same for basin index
Basin_1 = np.zeros_like(V_early)
Basin_2 = np.zeros_like(V_early)
Basin_names = []
for i in range(np.array(Basins).size):
    Basin_1[i,:] = i
    Basin_2[i,:] = i
    for j in range(V_early.shape[-1]):
        #... and for basin name
        Basin_names.append(Basins[i])

Basin_1_inds = Basin_1.flatten()
Basin_2_inds = Basin_2.flatten()

#... and for the edges of the BSP bins
## Here we calculate the mean TS edges averaged over both early and late times
S_start = (0.5*(Part_early.values[:,:,0]+Part_late.values[:,:,0])).flatten()-S0 # Remove reference salinity S0
S_end = (0.5*(Part_early.values[:,:,1]+Part_late.values[:,:,1])).flatten()-S0 # Remove reference salinity S0
T_start = (0.5*(Part_early.values[:,:,2]+Part_late.values[:,:,2])).flatten()
T_end = (0.5*(Part_early.values[:,:,3]+Part_late.values[:,:,3])).flatten()

# Any NaNs are zeroed out
S_1[np.isnan(S_1)] = 0
S_2[np.isnan(S_2)] = 0
T_1[np.isnan(T_1)] = 0
T_2[np.isnan(T_2)] = 0

In [5]:
## Here, we create the tracers and volumes matrices, which will be fed into the MTM function

volumes = np.stack((Vol_1, Vol_2), axis=0)/volnorming # Shape: [2 x N]

salinities = np.stack((S_1, S_2), axis=0)
temps = np.stack((T_1, T_2), axis=0)

tracers = np.stack((salinities, temps),axis=1) # Shape: [2 x M x N], where M = 2 for just T and S, and M>2 for T,S+other tracers

In [6]:
print('Total number of bins =', int(Vol_1.shape[0]))
N = int(Vol_1.shape[0])

Total number of bins = 1152


---------------
## C) Define Constraints

### i) Define Connectivity

We must define whether a BSP bin is allowed to transport volume to another BSP bin. In the simplest case, all bins are allowed to transport to one another - but this yields nonphysical transport across vast distances and TS bounds.
To improve on this, two connectivity constraints are used: 
1) Are the basins adjacent? This is defined via the connectivity array, below
2) If YES, do the BSP bins have overlapping (or the same) TS boundaries?\
If yes, the bins are connected. If no, they are not.


In [7]:
# Array defining the connection between the 9 basins;
# 1 = connected, 0 = disconnected
connectivity_array = np.zeros((Basins.size,Basins.size))

connectivity_array[0,:] = [1, 1, 0, 0, 0, 0, 0, 0, 0]
connectivity_array[1,:] = [1, 1, 1, 0, 0, 0, 0, 0, 0]
connectivity_array[2,:] = [0, 1, 1, 1, 0, 0, 0, 0, 0]
connectivity_array[3,:] = [0, 0, 1, 1, 0, 0, 0, 0, 1]
connectivity_array[4,:] = [0, 0, 0, 0, 1, 0, 1, 0, 1]
connectivity_array[5,:] = [0, 0, 0, 0, 0, 1, 1, 0, 1]
connectivity_array[6,:] = [0, 0, 0, 0, 1, 1, 1, 1, 0]
connectivity_array[7,:] = [0, 0, 0, 0, 0, 0, 1, 1, 0]
connectivity_array[8,:] = [0, 0, 0, 1, 1, 1, 0, 0, 1]

In [8]:
# Define whether a bin is connected to every other bin
# The two constraints used are: are the basins adjacent? 
# If yes, are the bin indices the same? 
# If yes, the bins are connected; if no, they are not connected. 
connected = np.zeros((Vol_1.size, Vol_1.size))

for i in tqdm(range(Vol_1.size)):
    for j in range(Vol_2.size):
        if connectivity_array[int(Basin_1_inds[i]), int(Basin_2_inds[j])]==1:
            if Basin_names[i] == Basin_names[j]:
                connected[i,j] = 1
            elif S_start[i]==S_start[j] and T_start[i]==T_start[j]:
                connected[i,j] = 1

constraints = connected # Shape: An [N x N] matrix

  0%|          | 0/1152 [00:00<?, ?it/s]

### ii) Define Weights

In [16]:
## We create a weight matrix
# Note that the code is very sensitive to the weights we input, and may break (TO BE DEVELOPED)
offset = 10**12 # To prevent divide by zero errors
weights = np.stack(((ST_scale)/(np.log10(A_2)),1/(np.log10(A_2))), axis=0) # Shape: An [M x N] matrix
weights = np.ones_like(weights)

1.0


-----------------
## D) Run Optimisation

In [17]:
## We run the optimiser to get the transports between water masses and the T,S mixed and T,S adjustment
result = MTM.optimise(tracers, volumes, constraints, weights)

g_ij = result['g_ij'] ## An [N x N] matrix of transports between WMs
Mixing = result['Mixing'] ## An [M x N] matrix of dtracer mixing for each WM
Adjustment = result['Adjustment'] ## An [M x N] matrix of dtracer adjustment for each WM

                                     CVXPY                                     
                                    v1.1.13                                    
(CVXPY) May 09 04:52:04 PM: Your problem has 149760 variables, 2 constraints, and 0 parameters.
(CVXPY) May 09 04:52:06 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 09 04:52:06 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 09 04:52:06 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 09 04:52:06 PM: Compiling problem (target solver=MOSEK).
(CVXPY) May 09 04:52:06 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

In [20]:
## Break down the Mixing and Adjustment matrices into their constituent tracers
dT_mixing = Mixing[1,:]
dS_mixing = Mixing[0,:]
dS_adj = Adjustment[0,:]
dT_adj = Adjustment[1,:]

----------
## E) Save Output 

In [21]:
da_dT_mixing = xr.DataArray(data = dT_mixing, dims = ["WM_number"],
                           coords=dict(WM_number = np.arange(0,N)),
                        attrs=dict(description="Temperature Mixing", units="\Delta K", variable_id="EN4 Tmix"))
da_dS_mixing = xr.DataArray(data = dS_mixing, dims = ["WM_number"],
                           coords=dict(WM_number = np.arange(0,N)),
                        attrs=dict(description="Salinity Mixing", units="\Delta g/kg", variable_id="EN4 Smix"))
da_dT_adjustment = xr.DataArray(data = dT_adj, dims = ["WM_number"],
                           coords=dict(WM_number = np.arange(0,N)),
                        attrs=dict(description="Temperature Adjustment", units="\Delta K", variable_id="EN4 Tadj"))
da_dS_adjustment = xr.DataArray(data = dS_adj, dims = ["WM_number"],
                           coords=dict(WM_number = np.arange(0,N)),
                        attrs=dict(description="Salinity Adjustment", units="\Delta g/kg", variable_id="EN4 Sadj"))
da_gij = xr.DataArray(data = g_ij*volnorming, dims = ["WM_initial", "WM_final"],
                           coords=dict(WM_initial = np.arange(0,N), WM_final = np.arange(0,N)),
                        attrs=dict(description="Volume transport", units="m^3", variable_id="EN4 Gij"))

## Create xarray DataSet that will hold all these DataArrays
ds_BSP = xr.Dataset()
ds_BSP['dT_mixing'] = da_dT_mixing
ds_BSP['dS_mixing'] = da_dS_mixing
ds_BSP['dT_adjustment'] = da_dT_adjustment
ds_BSP['dS_adjustment'] = da_dS_adjustment
ds_BSP['gij'] = da_gij

ds_BSP.to_netcdf('Optimal_result_EN4.nc')

In [17]:
EN4_3D_data = xr.open_mfdataset('~/UNSW_Work/EN4_Data/EN_data/EN4_CT*.nc')
EN4_3D_mask = xr.open_mfdataset('~/UNSW_Work/Min_Transform_Method/mask*')
# partitions_bins = EN4_data.Partitions_hist.isel(Time=slice(0,12)).mean('Time').isel(Basin=0).values
# EN4_3D_mask_numpy = EN4_3D_mask.__xarray_dataarray_variable__.values
# EN4_3D_SA_mean = EN4_3D_data.Abs_Sal.isel(time=slice(0,12)).mean('time')
# EN4_3D_CT_mean = EN4_3D_data.Cons_Temp.isel(time=slice(0,12)).mean('time')

In [36]:
T_Av_adj_2D = T_Av_adj.reshape(Basins.size, 128)
S_Av_adj_2D = S_Av_adj.reshape(Basins.size, 128)
T_Av_mix_2D = T_Av_mix.reshape(Basins.size, 128)
S_Av_mix_2D = S_Av_mix.reshape(Basins.size, 128)


In [37]:
masked_T_adj_grid_bsum =0
masked_T_adj_grid_sum = 0
for j in tqdm(range(128)):
    for k in range(Basins.size):
        masked_T_adj_grid = xr.where((EN4_3D_SA_mean>partitions_bins[j,0])&\
                                (EN4_3D_SA_mean<partitions_bins[j,1])&\
                                (EN4_3D_CT_mean>partitions_bins[j,2])&\
                                (EN4_3D_CT_mean<partitions_bins[j,3]), \
                                T_Av_adj_2D[k,j]*EN4_3D_mask_numpy[k,:,:,:], 0)
        masked_T_adj_grid_bsum = masked_T_adj_grid_bsum+masked_T_adj_grid
    masked_T_adj_grid_sum = masked_T_adj_grid_bsum+masked_T_adj_grid_sum

# masked_S_adj_grid_bsum =0
# masked_S_adj_grid_sum = 0
# for j in tqdm(range(128)):
#     for k in range(Basins.size):
#         masked_S_adj_grid = xr.where((EN4_3D_SA_mean>partitions_bins[j,0])&\
#                                 (EN4_3D_SA_mean<partitions_bins[j,1])&\
#                                 (EN4_3D_CT_mean>partitions_bins[j,2])&\
#                                 (EN4_3D_CT_mean<partitions_bins[j,3]), \
#                                 S_Av_adj_2D[k,j]*EN4_3D_mask_numpy[k,:,:,:], 0)
#         masked_S_adj_grid_bsum = masked_S_adj_grid_bsum+masked_S_adj_grid
#     masked_S_adj_grid_sum = masked_S_adj_grid_bsum+masked_S_adj_grid_sum

  0%|          | 0/128 [00:00<?, ?it/s]

NameError: name 'EN4_3D_SA_mean' is not defined

In [38]:
masked_T_mix_grid_bsum =0
masked_T_mix_grid_sum = 0
for j in tqdm(range(128)):
    for k in range(Basins.size):
        masked_T_mix_grid = xr.where((EN4_3D_SA_mean>partitions_bins[j,0])&\
                                (EN4_3D_SA_mean<partitions_bins[j,1])&\
                                (EN4_3D_CT_mean>partitions_bins[j,2])&\
                                (EN4_3D_CT_mean<partitions_bins[j,3]), \
                                T_Av_mix_2D[k,j]*EN4_3D_mask_numpy[k,:,:,:], 0)
        masked_T_mix_grid_bsum = masked_T_mix_grid_bsum+masked_T_mix_grid
    masked_T_mix_grid_sum = masked_T_mix_grid_bsum+masked_T_mix_grid_sum
# masked_T_mix_grid_sum = masked_T_mix_grid_sum/(T_Av_mix.size)

# masked_S_mix_grid_bsum =0
# masked_S_mix_grid_sum = 0
# for j in tqdm(range(128)):
#     for k in range(Basins.size):
#         masked_S_mix_grid = xr.where((EN4_3D_SA_mean>partitions_bins[j,0])&\
#                                 (EN4_3D_SA_mean<partitions_bins[j,1])&\
#                                 (EN4_3D_CT_mean>partitions_bins[j,2])&\
#                                 (EN4_3D_CT_mean<partitions_bins[j,3]), \
#                                 S_Av_mix_2D[k,j]*EN4_3D_mask_numpy[k,:,:,:], 0)
#         masked_S_mix_grid_bsum = masked_S_mix_grid_bsum+masked_S_mix_grid
#     masked_S_mix_grid_sum = masked_S_mix_grid_bsum+masked_S_mix_grid_sum
# masked_S_mix_grid_sum = masked_S_mix_grid_sum/(S_Av_mix.size)

  0%|          | 0/128 [00:00<?, ?it/s]

NameError: name 'EN4_3D_SA_mean' is not defined

In [ ]:
masked_T_adj_grid_DA = ((masked_T_adj_grid_sum*EN4_3D_data.dVol.isel(time=0)).sum('depth')/(EN4_3D_data.dVol.isel(time=0)).sum('depth')).values
# masked_S_adj_grid_DA = ((masked_S_adj_grid_sum*EN4_3D_data.dVol.isel(time=0)).sum('depth')/(EN4_3D_data.dVol.isel(time=0)).sum('depth')).values
masked_T_mix_grid_DA = (((masked_T_mix_grid_sum*EN4_3D_data.dVol.isel(time=0)).sum('depth')*EN4_3D_data.dVol.isel(time=0,depth=0))/(EN4_3D_data.dVol.isel(time=0)).sum('depth')).values
# masked_S_mix_grid_DA = ((masked_S_mix_grid_sum*EN4_3D_data.dVol.isel(time=0)).sum('depth')/(EN4_3D_data.dVol.isel(time=0)).sum('depth')).values

In [ ]:
plt.pcolormesh(masked_T_adj_grid_DA/44, vmin=-1, vmax=1, cmap=plt.cm.bwr)
plt.gca().add_patch(patches.Rectangle((0,0), 360, 180, color='black',zorder=0))
plt.colorbar()
plt.show()

# plt.pcolormesh(masked_S_adj_grid_DA/44*10, vmin=-0.001, vmax=0.001, cmap=plt.cm.bwr)
# plt.gca().add_patch(patches.Rectangle((0,0), 360, 180, color='black',zorder=0))
# plt.colorbar()
# plt.show()

plt.pcolormesh(masked_T_mix_grid_DA/44, vmin=-1, vmax=1, cmap=plt.cm.bwr)
plt.gca().add_patch(patches.Rectangle((0,0), 360, 180, color='black',zorder=0))
plt.colorbar()
plt.show()

plt.pcolormesh((masked_T_mix_grid_DA+masked_T_adj_grid_DA)/44, vmin=-1, vmax=1, cmap=plt.cm.bwr)
plt.gca().add_patch(patches.Rectangle((0,0), 360, 180, color='black',zorder=0))
plt.colorbar()
plt.show()

In [ ]:
np.nansum((masked_T_mix_grid_DA+masked_T_adj_grid_DA))
# plt.hist(masked_T_adj_grid_DA.flatten()/44, bins = np.linspace(-1,1,150))
# plt.hist(masked_T_mix_grid_DA.flatten()/44, bins = np.linspace(-1,1,150))
# plt.hist((masked_T_adj_grid_DA+masked_T_mix_grid_DA).flatten()/44, bins = np.linspace(-1,1,150))


In [ ]:
# Compute Eulerian change in temperature (just a simple difference)
dT_eul=np.nanmean(CTlate-CTearly,axis=-1)
# Compute depth integrated heat content (units: W/m^2)
dH_mix = Cp*rho*np.nansum(dT_mix*vol,axis=-1)/(darea*dyrs*yr2sec)
dH_eul = Cp*rho*np.nansum(dT_eul*vol,axis=-1)/(darea*dyrs*yr2sec)
dH_adj = Cp*rho*np.nansum(dT_adj*vol,axis=-1)/(darea*dyrs*yr2sec)
dH_eul[dH_eul==0]=np.nan
dH_mix[dH_mix==0]=np.nan
dH_adj[dH_adj==0]=np.nan
dH_redist = dH_eul-dH_mix-dH_adj

# Compare Mixing, Redistribution and Adjustment fresh water change
# Compute Eulerian change in temperature (just a simple difference)
dS_eul=np.nanmean(SAlate-SAearly,axis=-1)
# Compute implied fresh water content change (units: mm/yr)
dW_mix = (-1/S0)*np.nansum(dS_mix*vol,axis=-1)/(darea*dyrs)*1000
dW_eul = (-1/S0)*np.nansum(dS_eul*vol,axis=-1)/(darea*dyrs)*1000
dW_adj = (-1/S0)*np.nansum(dS_adj*vol,axis=-1)/(darea*dyrs)*1000
dW_eul[dW_eul==0]=np.nan
dW_mix[dW_mix==0]=np.nan
dW_adj[dW_adj==0]=np.nan
dW_redist = dW_eul - dW_mix - dW_adj

In [ ]:
fig, axs = plt.subplots(2,3, figsize=(40*0.5,17.5*0.5), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .15, wspace=0.15)
axs = axs.ravel() 
cax1 = fig.add_axes([0.925, 0.55, 0.02, 0.34])
cax2 = fig.add_axes([0.925, 0.12, 0.02, 0.34])

axs[0].add_patch(patches.Rectangle((np.min(lon), np.min(lat)), lon.size, lat.size, color='black',zorder=0))
axs[1].add_patch(patches.Rectangle((np.min(lon), np.min(lat)), lon.size, lat.size, color='black',zorder=0))
axs[2].add_patch(patches.Rectangle((np.min(lon), np.min(lat)), lon.size, lat.size, color='black',zorder=0))
axs[3].add_patch(patches.Rectangle((np.min(lon), np.min(lat)), lon.size, lat.size, color='black',zorder=0))
axs[4].add_patch(patches.Rectangle((np.min(lon), np.min(lat)), lon.size, lat.size, color='black',zorder=0))
axs[5].add_patch(patches.Rectangle((np.min(lon), np.min(lat)), lon.size, lat.size, color='black',zorder=0))

axs[0].pcolormesh(lon,lat,np.moveaxis(dH_mix, -1,0), vmin=-150,vmax=150, zorder=1,  cmap='bwr')
axs[1].pcolormesh(lon,lat,np.moveaxis(dH_redist, -1,0), vmin=-150, vmax=150, zorder=1, cmap='bwr')
axs[2].pcolormesh(lon,lat,np.moveaxis(dH_adj, -1,0), vmin=-150, vmax=150, zorder=1, cmap='bwr')
axs[3].pcolormesh(lon,lat,np.moveaxis(dW_mix, -1,0), vmin=-5000,vmax=5000, zorder=1,  cmap='bwr')
axs[4].pcolormesh(lon,lat,np.moveaxis(dW_redist, -1,0), vmin=-5000, vmax=5000, zorder=1, cmap='bwr')
axs[5].pcolormesh(lon,lat,np.moveaxis(dW_adj, -1,0), vmin=-5000, vmax=5000, zorder=1, cmap='bwr')

axs[0].grid(True, zorder=2)
axs[1].grid(True, zorder=2)
axs[2].grid(True, zorder=2)
axs[3].grid(True, zorder=2)
axs[4].grid(True, zorder=2)
axs[5].grid(True, zorder=2)

axs[0].set_title('Mixing')
axs[1].set_title('Adiabatic Redistribution')
axs[2].set_title(r'Adjustment')

axs[3].set_xlabel('Longitude')
axs[4].set_xlabel('Longitude')
axs[5].set_xlabel('Longitude')

axs[0].set_ylabel('Latitude')
axs[3].set_ylabel('Latitude')

axs[0].set_xlim(np.min(lon), np.max(lon))
axs[1].set_xlim(np.min(lon), np.max(lon))
axs[2].set_xlim(np.min(lon), np.max(lon))
axs[3].set_xlim(np.min(lon), np.max(lon))
axs[4].set_xlim(np.min(lon), np.max(lon))
axs[5].set_xlim(np.min(lon), np.max(lon))

axs[0].set_ylim(np.min(lat), np.max(lat))
axs[1].set_ylim(np.min(lat), np.max(lat))
axs[2].set_ylim(np.min(lat), np.max(lat))
axs[3].set_ylim(np.min(lat), np.max(lat))
axs[4].set_ylim(np.min(lat), np.max(lat))
axs[5].set_ylim(np.min(lat), np.max(lat))

m = plt.cm.ScalarMappable(cmap='bwr')
m.set_clim(-5,5)
cbar2 = fig.colorbar(m, cax=cax2, orientation='vertical')
cbar2.set_label('$m/yr$') 

m = plt.cm.ScalarMappable(cmap='bwr')
m.set_clim(-150,150)
cbar1 = fig.colorbar(m, cax=cax1, orientation='vertical')
cbar1.set_label('$W/m^2$') 

plt.savefig('Minimisation_remapped_128_unmixing.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(30*0.5,10*0.5), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .15, wspace=0.15)
axs = axs.ravel() 

ind = np.arange(4)
width=1/6
axs[0].barh(ind+width/2*3, \
        [dH_mix_Atl_avg , dH_adj_Atl_avg , dH_redist_Atl_avg, dH_redist_Atl_avg+dH_mix_Atl_avg+dH_adj_Atl_avg], width, color='skyblue', label='Atlantic')
axs[0].barh(ind+width/2, \
        [dH_mix_SO_avg, dH_adj_SO_avg,  dH_redist_SO_avg, dH_redist_SO_avg+dH_mix_SO_avg+dH_adj_SO_avg], width, color='orange', label='Southern')
axs[0].barh(ind-width/2, \
        [dH_mix_Pac_avg, dH_adj_Pac_avg,  dH_redist_Pac_avg, dH_redist_Pac_avg+dH_mix_Pac_avg+dH_adj_Pac_avg], width, color='purple', label='Pacific')
axs[0].barh(ind-3*width/2, \
        [dH_mix_Ind_avg,  dH_adj_Ind_avg, dH_redist_Ind_avg, \
         dH_redist_Ind_avg+dH_mix_Ind_avg+dH_adj_Ind_avg], width, color='black', label='Indian')

axs[1].barh(ind+width/2*3, \
        [dW_mix_Atl_avg , dW_adj_Atl_avg , dW_redist_Atl_avg, dW_redist_Atl_avg+dW_mix_Atl_avg+dW_adj_Atl_avg], width, color='skyblue', label='Atlantic')
axs[1].barh(ind+width/2, \
        [dW_mix_SO_avg, dW_adj_SO_avg,  dW_redist_SO_avg, dW_redist_SO_avg+dW_mix_SO_avg+dW_adj_SO_avg], width, color='orange', label='Southern')
axs[1].barh(ind-width/2, \
        [dW_mix_Pac_avg, dW_adj_Pac_avg,  dW_redist_Pac_avg, dW_redist_Pac_avg+dW_mix_Pac_avg+dW_adj_Pac_avg], width, color='purple', label='Pacific')
axs[1].barh(ind-3*width/2, \
        [dW_mix_Ind_avg,  dW_adj_Ind_avg, dW_redist_Ind_avg, \
         dW_redist_Ind_avg+dW_mix_Ind_avg+dW_adj_Ind_avg], width, color='black', label='Indian')


axs[0].grid()
# axs[0].set_ylabel('Basin', labelpad=25)
axs[0].set_xlabel('Mean Heat Content Change [Wm$^{-2}$]')
axs[0].set_yticks([0,1,2,3])
axs[0].set_yticklabels(['Mixing', 'Adjustment', 'Redistribution', 'Total'])
axs[0].set_ylim(ind[0]-width*2, ind[-1]+width*2)
axs[0].legend(ncol=4, bbox_to_anchor=(1.05,-0.3), loc='center')
# axs[0].set_xlim(-3.1,3.1)

axs[1].grid()
axs[1].set_xlabel('Mean Freshwater Change [myr$^{-1}$]')
axs[1].set_yticks([0,1,2,3])
axs[1].set_yticklabels(['','','',''])
axs[1].set_ylim(ind[0]-width*2, ind[-1]+width*2)
# axs[1].set_xlim(-0.08,0.08)

plt.savefig('Minimisation_basin_bars_global.png', dpi=300, bbox_inches='tight')
plt.show()